In [10]:
import h5py
import pynwb
import pynapple as nap
import dendro.client as prc
import remfile
from TuningCurves2DView import TuningCurves2DView


# Load project D-000582
project = prc.load_project("a7852166")
dandiset_id = "000582"

# Select an NWB file
asset_path = "sub-10073/sub-10073_ses-17010302_behavior+ecephys.nwb"

# Lazy load NWB file
file = remfile.File(project.get_file(f"imported/{dandiset_id}/{asset_path}"))
io = pynwb.NWBHDF5IO(file=h5py.File(file, "r"), mode="r")
nwbfile = io.read()

In [11]:
# Load the spatial series into a pynapple TsdFrame
spatial_series = nwbfile.processing["behavior"]["Position"]["SpatialSeriesLED1"]
position_over_time = nap.TsdFrame(
    d=spatial_series.data[:],
    t=spatial_series.timestamps[:],
    columns=["x", "y"],
)

# Load the unit spike times into a pynapple TsGroup
unit_names = nwbfile.units["unit_name"][:]
unit_spike_times = nwbfile.units["spike_times"][:]
spike_times_group = nap.TsGroup(
    {i: unit_spike_times[i] for i in range(len(unit_names))}
)

# Compute 2D tuning curves
num_bins = 30
rate_maps, position_bins = nap.compute_2d_tuning_curves(
    spike_times_group,
    position_over_time,
    num_bins,
)

/tmp/ipykernel_20458/1564342580.py:12: UserWarning: Elements should not be passed as numpy array. Default time units is seconds when creating the Ts object.
  spike_times_group = nap.TsGroup(
/home/magland/miniconda3/envs/dev/lib/python3.9/site-packages/pynapple/process/tuning_curves.py:203: RuntimeWarning: invalid value encountered in divide
  count = count / occupancy


In [12]:
# Construct a tuning curves figurl view
V = TuningCurves2DView(
    rate_maps=[rate_maps[i] for i in range(len(unit_names))],
    x_bin_positions=position_bins[0],
    y_bin_positions=position_bins[1],
    unit_ids=unit_names,
    unit_num_spikes=[len(spike_times_group[i]) for i in range(len(unit_names))],
)
view_url = V.get_view_url(label=f"{dandiset_id}/{asset_path}|tuning-curves-2d")

# Print the view URL
print(view_url)

# Display the view in an iframe
from IPython.display import IFrame
display(IFrame(view_url + "&hide=1", width="95%", height=500))

https://figurl.org/f?v=https://figurl-tuning-curves-1.surge.sh&d=sha1://ee7e7b9368aee938bbafe840fb9b21f3f7ca89aa&label=000582/sub-10073/sub-10073_ses-17010302_behavior+ecephys.nwb|tuning-curves-2d
